In [1]:
import duckdb

In [2]:
duckdb.sql("SELECT 42").show()

┌───────┐
│  42   │
│ int32 │
├───────┤
│    42 │
└───────┘



In [3]:
duckdb.read_csv("fact_player_stats.csv") # read a CSV file into a Relation

┌────────┬─────────────┬───────────┬───────────┬───────────────┬─────────┬───────────┬────────────┐
│ fps_id │ opponent_id │ games_won │ season_id │ match_date_id │   h_a   │ player_id │ games_lost │
│ int64  │    int64    │   int64   │   int64   │     int64     │ varchar │   int64   │   int64    │
├────────┼─────────────┼───────────┼───────────┼───────────────┼─────────┼───────────┼────────────┤
│      1 │           5 │         2 │         1 │          2172 │ H       │         4 │          0 │
│      2 │           4 │         1 │         1 │          2179 │ A       │         4 │          0 │
│      3 │           9 │         0 │         1 │          2214 │ A       │         4 │          1 │
│      4 │           2 │         0 │         1 │          2221 │ A       │         4 │          1 │
│      5 │           8 │         1 │         1 │          2228 │ H       │         4 │          1 │
│      6 │           3 │         2 │         1 │          2235 │ A       │         4 │          0 │


In [6]:
# create a connection to a file called 'file.db'
con = duckdb.connect("pool_stats.db")

In [14]:
opp_history = con.sql(
    """
select t.team_name as teamname, h_a as home_away, count(distinct match_date_id) as total_played, 
        sum(games_won) as games_won, sum(games_lost) as games_lost 
from fact_player_stats f, 
     dim_season s, 
     dim_team t 
where f.season_id = s.season_id 
and t.team_id = f.opponent_id 
group by t.team_name, h_a 
order by games_won desc, team_name
""").df()

In [17]:
opp_history.head(20)

,teamname,home_away,total_played,games_won,games_lost
0,Garibaldi,A,7,32.0,20.0
1,Garibaldi,H,5,22.0,20.0
2,The Crown (Hardingstone),A,5,19.0,21.0
3,Road To Morroco,H,4,18.0,10.0
4,Squirrels,A,4,18.0,13.0
5,Cock Inn A,H,6,17.0,17.0
6,Barratts A,H,4,16.0,14.0
7,Coopers Q,A,5,16.0,16.0
8,Coopers Q,H,3,16.0,14.0
9,Cock Inn A,A,5,15.0,13.0


In [26]:
players_streaks = con.sql(
    """
select p.player_name, h_a, games_won, games_lost, d.CalendarDateInterval
from fact_player_stats f, dim_player p, DateDimension d
where p.player_id = f.player_id
and f.match_date_id = d.date_id
order by p.player_name, d.CalendarDateInterval desc
""").df()

In [27]:
players_streaks.head(10)

,player_name,h_a,games_won,games_lost,CalendarDateInterval
0,Colin Chapman,H,0,1,"[50, 48, 50, 51, 45, 48, 52, 45, 49, 55, 32, 4..."
1,Colin Chapman,H,1,0,"[50, 48, 50, 51, 45, 48, 52, 45, 49, 48, 32, 4..."
2,Colin Chapman,H,1,0,"[50, 48, 50, 51, 45, 48, 52, 45, 48, 51, 32, 4..."
3,Colin Chapman,A,0,1,"[50, 48, 50, 51, 45, 48, 51, 45, 50, 55, 32, 4..."
4,Colin Chapman,H,0,1,"[50, 48, 50, 51, 45, 48, 51, 45, 50, 48, 32, 4..."
5,Colin Chapman,A,1,0,"[50, 48, 50, 51, 45, 48, 51, 45, 49, 51, 32, 4..."
6,Colin Chapman,A,1,0,"[50, 48, 50, 51, 45, 48, 51, 45, 48, 54, 32, 4..."
7,Colin Chapman,A,1,0,"[50, 48, 50, 51, 45, 48, 50, 45, 50, 55, 32, 4..."
8,Colin Chapman,H,0,1,"[50, 48, 50, 51, 45, 48, 50, 45, 50, 48, 32, 4..."
9,Colin Chapman,A,1,0,"[50, 48, 50, 51, 45, 48, 50, 45, 49, 51, 32, 4..."


In [28]:
opps_all_hist = con.sql(
    """
select  t.team_name, count(distinct match_date_id) as "total_played", 
        sum(games_won) as "games_won", sum(games_lost) as "games_lost"
from fact_player_stats f,
     dim_season s,
     dim_team t
where f.season_id = s.season_id
and t.team_id = f.opponent_id
group by t.team_name
order by games_won desc, team_name
""").df()

In [30]:
opps_all_hist.head(20)

,team_name,total_played,games_won,games_lost
0,Garibaldi,12,54.0,40.0
1,Cock Inn A,10,32.0,30.0
2,Coopers Q,8,32.0,30.0
3,Squirrels,6,32.0,29.0
4,The Crown (Hardingstone),8,31.0,37.0
5,Barratts A,8,28.0,34.0
6,King Billy A,4,27.0,13.0
7,Road To Morroco,8,27.0,32.0
8,Sevens,4,25.0,15.0
9,Forresters,7,24.0,36.0


In [31]:
players_rec_teams = con.sql(
    """
select  p.player_name, 
        team_name, 
        sum(games_won) won, 
        sum(games_lost) lost,
        count(distinct match_date_id) matches
 from fact_player_stats f,
      dim_team t,
      dim_player p
 where t.team_id = f.opponent_id
 and p.player_id = f.player_id
 group by p.player_name, team_name
 order by p.player_name, team_name
""").df()

In [32]:
players_rec_teams.head(20)

,player_name,team_name,won,lost,matches
0,Colin Chapman,Barratts A,2.0,2.0,4
1,Colin Chapman,Barratts B,0.0,3.0,2
2,Colin Chapman,ClubSports A,2.0,1.0,2
3,Colin Chapman,ClubSports C,1.0,0.0,1
4,Colin Chapman,Cock Inn A,1.0,3.0,4
5,Colin Chapman,Coopers B,1.0,2.0,2
6,Colin Chapman,Coopers Q,5.0,5.0,6
7,Colin Chapman,Coopers Y,3.0,1.0,2
8,Colin Chapman,Edge of Town,2.0,2.0,2
9,Colin Chapman,Forresters,2.0,4.0,6


In [33]:
players_vs_opps = con.sql(
    """
select  p.player_name, team_name, count(distinct match_date_id) as "total_played",
       sum(games_won) as "games_won", sum(games_lost) as "games_lost"
from fact_player_stats f,
     dim_team t,
     dim_player p
where t.team_id = f.opponent_id
and p.player_id = f.player_id
/* and p.player_name = 'Liam Chapman' */
group by p.player_name, team_name
order by p.player_name, team_name
""").df()

In [34]:
players_vs_opps

,player_name,team_name,total_played,games_won,games_lost
0,Colin Chapman,Barratts A,4,2.0,2.0
1,Colin Chapman,Barratts B,2,0.0,3.0
2,Colin Chapman,ClubSports A,2,2.0,1.0
3,Colin Chapman,ClubSports C,1,1.0,0.0
4,Colin Chapman,Cock Inn A,4,1.0,3.0
...,...,...,...,...,...
209,Steve Dunkley,Swan & Helmet,2,1.0,1.0
210,Steve Dunkley,The Bear,2,1.0,1.0
211,Steve Dunkley,The Crown (Hardingstone),1,0.0,1.0
212,Steve Dunkley,Turnpike,1,0.0,2.0


In [36]:
seasons = con.sql(
    """
select  *  from dim_season
""").df()

In [37]:
seasons

,season_id,season_name
0,1,"[68, 101, 99, 32, 50, 48, 50, 50, 32, 45, 32, ..."
1,2,"[74, 117, 110, 32, 50, 48, 50, 50, 32, 45, 32,..."
2,3,"[83, 101, 112, 32, 50, 48, 49, 56, 32, 45, 32,..."
3,4,"[74, 117, 110, 32, 32, 50, 48, 49, 57, 32, 45,..."
4,5,"[78, 111, 118, 32, 50, 48, 50, 49, 32, 45, 32,..."
